## Model training

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [5]:
df=pd.read_csv("data\gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [6]:
df=df.drop(labels=["id"],axis=1)

In [7]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [8]:
x=df.drop(labels=["price"],axis=1)
y=df[["price"]]

In [9]:
categorical_cols=x.select_dtypes(include='object').columns
numerical_cols=x.select_dtypes(exclude='object').columns

In [10]:
categorical_cols,numerical_cols

(Index(['cut', 'color', 'clarity'], dtype='object'),
 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object'))

In [11]:
cut_categories=['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
color_categories=['F', 'J', 'G', 'E', 'D', 'H', 'I']
clarity_categories=['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']

In [12]:
from sklearn.impute import SimpleImputer # Handling missing values 
from sklearn.preprocessing import StandardScaler # Handling feature scaling
from sklearn.preprocessing import OrdinalEncoder # for ordinal encoder whenever our categorical features has rank we have to use ordinal enocder
## pipelines
from sklearn.pipeline import Pipeline # for connecting 
from sklearn.compose import ColumnTransformer # for formulating all.


In [13]:
## Numerical pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

## Categorical pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinal',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols),
])

In [14]:
## Train test split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=42)

In [26]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out()) # to get teh columns names we need to use get function. this is just to have the columns names 
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out()) # to get teh columns names we need to use get function. this is just to have the columns names 


In [27]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.652424,0.058753,1.605394
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-0.381506,1.123070,-0.079246
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-1.415437,1.123070,-1.202339
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.652424,-1.537723,2.166941
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-0.381506,1.655229,0.482301


In [29]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.629077,0.258230,-0.120630,-0.600482,-0.581521,-0.572248,0.652424,0.590912,0.482301
1,2.605374,-2.148014,-0.120630,2.126042,2.198832,1.959219,-0.381506,-0.473406,-0.640792
2,-1.125026,-1.222536,0.921902,-1.374347,-1.414721,-1.469110,-1.415437,0.058753,2.166941
3,-1.017211,-0.574701,0.921902,-1.158385,-1.161138,-1.194265,-1.415437,1.655229,2.166941
4,0.858771,0.628421,-0.641897,0.947248,0.985258,1.004495,0.652424,1.123070,0.482301


In [30]:
print(X_train.shape,X_test.shape)

(135501, 9) (58072, 9)


In [33]:
print(X_train.isnull().sum())
print("\n")
print(X_test.isnull().sum())

num_pipeline__carat      0
num_pipeline__depth      0
num_pipeline__table      0
num_pipeline__x          0
num_pipeline__y          0
num_pipeline__z          0
cat_pipeline__cut        0
cat_pipeline__color      0
cat_pipeline__clarity    0
dtype: int64


num_pipeline__carat      0
num_pipeline__depth      0
num_pipeline__table      0
num_pipeline__x          0
num_pipeline__y          0
num_pipeline__z          0
cat_pipeline__cut        0
cat_pipeline__color      0
cat_pipeline__clarity    0
dtype: int64


In [34]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [35]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [36]:
regression.coef_

array([[ 6538.82211476,  -232.4061404 ,  -177.05602534, -2090.53215704,
         -556.05883268,   -44.19321465,   -10.72128479,  -181.811724  ,
          121.17907723]])

In [37]:
regression.intercept_

array([3976.8787389])

In [48]:
import numpy as np 
def evaluate_model(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2_square = r2_score(y_test, y_pred)
    return mae, rmse, r2_square

In [60]:
# Define a dictionary of models
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet()
}

# Initialize lists to store model names and R-squared scores
trained_model_list=[]
model_list = []
r2_list = []

# Train and evaluate each model
for model_name, model in models.items():
    try:
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate model performance
        mae, rmse, r2_square = evaluate_model(y_test, y_pred)

        # Print model performance metrics
        print(f"Model: {model_name}")
        print(f"Model Training Performance for {model_name}:")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        r2_square=r2_square*100     # To get the result in percentage
        print(f"R2 score: {r2_square:.4f}")

        # Append model name and R-squared score to lists
        model_list.append(model_name)
        trained_model_list.append(r2_square)
        r2_list.append(r2_square)
        print('='*35)
        print("\n")

    except Exception as e:
        print(f"Error training {model_name}: {str(e)}")




Model: LinearRegression
Model Training Performance for LinearRegression:
RMSE: 1196.8249
MAE: 723.6035
R2 score: 91.1356


Model: Lasso
Model Training Performance for Lasso:
RMSE: 1196.7937
MAE: 725.0551
R2 score: 91.1361


Model: Ridge
Model Training Performance for Ridge:
RMSE: 1196.8308
MAE: 723.6371
R2 score: 91.1355


Model: ElasticNet
Model Training Performance for ElasticNet:
RMSE: 1632.5394
MAE: 1135.0836
R2 score: 83.5064




In [61]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

In [62]:
trained_model_list

[91.13560571385322, 91.13606781270963, 91.13551825603494, 83.5064099195437]